In [27]:
import torch
import torch.nn as nn
import torch.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
import os
import glob
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import nibabel as nib

In [8]:
nii_list=os.listdir("/kaggle/input/mri-preprocessed/pre_processed_files1")

# print(len(list))

shape=(0,0)
for i in range(1,len(list)):
    nii_img= nib.load(os.path.join("/kaggle/input/mri-preprocessed/pre_processed_files1",list[i]))
    nii_data= nii_img.get_fdata()
    
    

<class 'tuple'>


In [8]:
import nibabel as nib
import os
import numpy as np
import matplotlib.pyplot as plt

# Directory path
data_dir = "/kaggle/input/mri-preprocessed/pre_processed_files1"
nii_list = os.listdir(data_dir)

max_axial_shape = (0, 0)

for filename in nii_list:
    path = os.path.join(data_dir, filename)
    nii_img = nib.load(path)
    data = nii_img.get_fdata()
    
    axial_shape = data.shape[:2]

    if np.prod(axial_shape) > np.prod(max_axial_shape):
        max_axial_shape = axial_shape

print(f"Axial plane shape: {max_axial_shape}")

Axial plane shape: (184, 256)


In [12]:
def pad_to_shape(img, target_shape):
    h, w = img.shape
    H, W = target_shape
    pad_top = (H - h) // 2
    pad_bottom = H - h - pad_top
    pad_left = (W - w) // 2
    pad_right = W - w - pad_left
    return np.pad(img, ((pad_top, pad_bottom), (pad_left, pad_right)), mode='constant')

clahe= cv2.createCLAHE(clipLimit=6.0, tileGridSize=(4, 4))

def apply_clahe(img):
    
    img= cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX)
    clahe= cv2.createCLAHE(clipLimit=6.0, tileGridSize=(4, 4))
    img_uint8= img.astype(np.uint8)
    
    # Apply CLAHE
    clahe_img= clahe.apply(img_uint8)
    
    return clahe_img / 255.0
    
padded_images = []

for file in os.listdir(data_dir):
    if file.endswith('.nii') or file.endswith('.nii.gz'):
        nii_path = os.path.join(data_dir, file)
        data = nib.load(nii_path).get_fdata()

        # Extract middle axial slice
        axial = data[:, :, data.shape[2] // 2]

        # Apply CLAHE
        clahe_img = apply_clahe(axial)

        # Pad to max_axial_shape
        padded = pad_to_shape(clahe_img, max_axial_shape)

        padded_images.append(padded)

# Convert to NumPy array
X = np.array(padded_images).reshape(-1, max_axial_shape[0], max_axial_shape[1], 1)
print(f"X shape: {X.shape}")

X shape: (445, 184, 256, 1)


In [24]:
df= pd.read_csv("/kaggle/input/mri-preprocessed/ADNI1_Complete_1Yr_1.5T_6_20_2025.csv")

nii_list= os.listdir("/kaggle/input/mri-preprocessed/pre_processed_files1")

img_id=[]
for file in nii_list:
    img_id.append(file.split('_')[-3])

matched_df = df[df['Image Data ID'].isin(img_id)]

Grp= matched_df['Group'].tolist()

y=[]
for grp in Grp:
    if grp=="CN":
        y.append(1)
    else:
        y.append(0)

print(len(y))

y=np.array(y)
print(type(y))

445
<class 'numpy.ndarray'>


In [36]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, losses
from tensorflow.keras.utils import to_categorical

input_shape = (184, 256, 1)  # HWC format for TensorFlow

# cnn1 block
def cnn1_block(input_layer):
    x = layers.Conv2D(16, kernel_size=3, padding='same', activation='relu')(input_layer)
    x = layers.Conv2D(16, kernel_size=3, padding='same', activation='relu')(x)
    x = layers.MaxPooling2D(pool_size=2)(x)

    x = layers.Conv2D(64, kernel_size=3, padding='same', activation='relu')(x)
    x = layers.Conv2D(64, kernel_size=3, padding='same', activation='relu')(x)
    x = layers.MaxPooling2D(pool_size=2)(x)

    x = layers.Conv2D(256, kernel_size=3, padding='same', activation='relu')(x)
    x = layers.MaxPooling2D(pool_size=2)(x)

    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Flatten()(x)

    x = layers.Dense(128)(x)
    return x

# cnn2 block
def cnn2_block(input_layer):
    x = layers.Conv2D(32, kernel_size=5, padding='same', activation='relu')(input_layer)
    x = layers.Conv2D(32, kernel_size=5, padding='same', activation='relu')(x)
    x = layers.MaxPooling2D(pool_size=2)(x)

    x = layers.Conv2D(128, kernel_size=5, padding='same', activation='relu')(x)
    x = layers.Conv2D(128, kernel_size=5, padding='same', activation='relu')(x)
    x = layers.MaxPooling2D(pool_size=2)(x)

    x = layers.Conv2D(512, kernel_size=5, padding='same', activation='relu')(x)
    x = layers.MaxPooling2D(pool_size=2)(x)

    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Flatten()(x)

    x = layers.Dense(128)(x)
    return x

# Combine CNN1 and CNN2
inputs = tf.keras.Input(shape=input_shape)

x1 = cnn1_block(inputs)
x2 = cnn2_block(inputs)

x = layers.Concatenate()([x1, x2])
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(optimizer=optimizers.Adam(1e-4),
              loss=losses.BinaryCrossentropy(),
              metrics=['accuracy'])

model.summary()


2025-06-26 12:43:12.613484: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750941792.931803      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750941793.019012      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
I0000 00:00:1750941807.460471      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1750941807.462783      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability:

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 184, 256, 1)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 184, 256, 16)   │            160 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_5 (Conv2D)         │ (None, 184, 256, 32)   │            832 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 184, 256, 16)   │          2,320 │ conv2d[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_6 (Conv2D)         │ (None, 184, 256, 32)   │         25,632 │ conv2d_5[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d             │ (None, 92, 128, 16)    │              0 │ conv2d_1[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_3           │ (None, 92, 128, 32)    │              0 │ conv2d_6[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2 (Conv2D)         │ (None, 92, 128, 64)    │          9,280 │ max_pooling2d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_7 (Conv2D)         │ (None, 92, 128, 128)   │        102,528 │ max_pooling2d_3[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_3 (Conv2D)         │ (None, 92, 128, 64)    │         36,928 │ conv2d_2[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_8 (Conv2D)         │ (None, 92, 128, 128)   │        409,728 │ conv2d_7[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_1           │ (None, 46, 64, 64)     │              0 │ conv2d_3[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_4           │ (None, 46, 64, 128)    │              0 │ conv2d_8[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_4 (Conv2D)         │ (None, 46, 64, 256)    │        147,712 │ max_pooling2d_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_9 (Conv2D)         │ (None, 46, 64, 512)    │      1,638,912 │ max_pooling2d_4[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_2           │ (None, 23, 32, 256)    │              0 │ conv2d_4[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_5           │ (None, 23, 32, 512)    │              0 │ conv2d_9[0][0]         │
│ (MaxPooling2D)       

 Total params: 74,762,129 (285.19 MB)

 Trainable params: 74,760,593 (285.19 MB)

 Non-trainable params: 1,536 (6.00 KB)

In [38]:
history= model.fit(X, y, validation_split=0.2, epochs=10, batch_size=32)

Epoch 1/10


I0000 00:00:1750942036.219487     118 service.cc:148] XLA service 0x7fee48008b60 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1750942036.220261     118 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1750942036.220297     118 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1750942037.108945     118 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-06-26 12:47:27.348009: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng12{k11=0} for conv (f32[32,128,92,128]{3,2,1,0}, u8[0]{0}) custom-call(f32[32,128,92,128]{3,2,1,0}, f32[128,128,5,5]{3,2,1,0}, f32[128]{0}), window={size=5x5 pad=2_2x2_2}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kNone","conv_result_scale":1,"leakyrelu_alpha":0,"side_input_scale":0},"force_earliest_s

11/12 ━━━━━━━━━━━━━━━━━━━━ 0s 386ms/step - accuracy: 0.6659 - loss: 1.0877

E0000 00:00:1750942100.170194     121 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1750942100.497823     121 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.


12/12 ━━━━━━━━━━━━━━━━━━━━ 115s 5s/step - accuracy: 0.6641 - loss: 1.0606 - val_accuracy: 0.6404 - val_loss: 0.6864
Epoch 2/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 371ms/step - accuracy: 0.7328 - loss: 0.5672 - val_accuracy: 0.6404 - val_loss: 0.6877
Epoch 3/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 368ms/step - accuracy: 0.7426 - loss: 0.5264 - val_accuracy: 0.6404 - val_loss: 0.6873
Epoch 4/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 370ms/step - accuracy: 0.7498 - loss: 0.5045 - val_accuracy: 0.6404 - val_loss: 0.6847
Epoch 5/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 371ms/step - accuracy: 0.7616 - loss: 0.4539 - val_accuracy: 0.6404 - val_loss: 0.6836
Epoch 6/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 374ms/step - accuracy: 0.8387 - loss: 0.3802 - val_accuracy: 0.6517 - val_loss: 0.6811
Epoch 7/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 5s 378ms/step - accuracy: 0.8930 - loss: 0.2871 - val_accuracy: 0.4494 - val_loss: 0.6931
Epoch 8/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 5s 379ms/step - accuracy: 0.8196 - loss: 0.3777 - val_accuracy: 0.6067 - val_l

In [39]:
model.save("ADNet_1.keras")